# Part 1: ID3


### Author: Yifan Wang

In [1]:
from sklearn.datasets import load_breast_cancer
import numpy as np
from collections import Counter

In [2]:
class id3_tree():
    'Implementation of ID3 Decision Tree in Python, majorly in NumPy'
    def __init__(self,least_children_num,verbose=True):
        self.least_children_num = least_children_num
        self.verbose = verbose
        
    
    def fit(self,tmp_x,tmp_y):
        def fit_tree(tmp_x,tmp_y):
        #     Exit Condition 0:
            # Exit Condition 1:
            if \
            len(tmp_y) < self.least_children_num or len(np.unique(tmp_y))==1:

                if self.verbose:
                    print('exit condition:')
                    print('tmp_y:')
                    print(tmp_y)

                mode_val = self.mode(tmp_y.flatten().tolist())
                return([np.nan, mode_val, np.nan, np.nan]) # Leaf Node: format [feat,splitval,]

            # Otherwise Split:
            if self.verbose:
                print("start....subset Y len {}".format(len(tmp_y)))


            split_row,split_col = self.decide_split_data(tmp_x,tmp_y)

            if not split_row and not split_col:
                print('no better split...return mode')
                mode_val = self.mode(tmp_y.flatten().tolist())
                return([np.nan, mode_val, np.nan, np.nan])

            if self.verbose:
                print("split on:")
                print(split_row,split_col)


            split_vec = tmp_x[:,split_col]
            split_val = tmp_x[split_row,split_col]

            # Recursively Split to left and right branches:
            left_ind = np.where(split_vec<split_val)[0].tolist()
            right_ind = np.where(split_vec>=split_val)[0].tolist()
            left_dat,left_y = tmp_x[left_ind,:],tmp_y[left_ind,]
            right_dat,right_y = tmp_x[right_ind,:],tmp_y[right_ind,]

            left_tree = fit_tree(left_dat,left_y)
            right_tree = fit_tree(right_dat,right_y)

            if isinstance(left_tree, list): # If list, tree len 1
                len_l_tree = 1
            else:
                len_l_tree = left_tree.shape[0] # If array, tree len >1

            root = [split_col,split_val,1,len_l_tree+1] # Format [split_col, split_val, left_tree_relative_idx, right_tree_relative_idx]
            return(np.vstack([root,left_tree,right_tree]))
        
        tree = fit_tree(tmp_x,tmp_y)
        self.tree = tree

    

    def decide_split_data(self,x,y):
        'Given subset of X,Y, search for the best splitting node based on: information gain'
        def entropy(tmp_y):
            'Key Metrics of building a decision tree. Specifically Shannon Entropy'
            tmp_ent = 0
            for uni_y in np.unique(tmp_y):
                p = len(tmp_y[tmp_y==uni_y])/len(tmp_y)
                tmp_ent -= (p*np.log2(p))
            return tmp_ent

        m,n = x.shape
        best_gain = 0
        split_row, split_col = None,None

        previous_entropy = entropy(y)
        for col in range(n):
            tmp_vec = x[:,col].ravel()

            for row in range(m):
                val = tmp_vec[row]
                # >= & < is my convention here:
                if val!=np.max(tmp_vec) and val!= np.min(tmp_vec):
                    left_b = np.where(tmp_vec<val)[0].tolist()
                    right_b = np.where(tmp_vec>=val)[0].tolist()

                    # new entropy is the weighted  average entropy from each of the subset
                    new_ent = \
                    (len(y[left_b])/len(y))*entropy(y[left_b]) + \
                    (len(y[right_b])/len(y))*entropy(y[right_b])


    #                 print('new entropy: %f'%new_ent)
                    info_gain = previous_entropy - new_ent

                    if info_gain > best_gain:
                        split_row, split_col = row,col
                        best_gain = info_gain
                        if self.verbose:
                            print('better gain:{}'.format(best_gain))
                            print()

        return split_row, split_col
                
                

    def mode(self, x_list):
        'calculate the mode'
        return Counter(x_list).most_common(1)[0][0]

    
    

    
    
    

    
    
    
    def predict(self, tmp_test_array):
        'Wrap-up fun for prediction'
        def query(tree,tmp_test_array):
            'Test for single example'
            assert len(tmp_test_array.shape) == 2, "Make sure your test data is 2d array"
        #     print(tree)

            start_node = tree[0,:] # Iteratively hit first row
        #     print(start_node)
        #     print()
            test_feat,test_val,left_tree_jump,right_tree_jump = start_node[0],start_node[1],start_node[2],start_node[3]

            # Exit Condition:
            if np.isnan(test_feat) and np.isnan(left_tree_jump) and np.isnan(right_tree_jump):
                pred = test_val;
                return pred 
            #Test:
            if tmp_test_array[0,int(test_feat)] < test_val:
                # If <, go left branch:
                jump_loc = left_tree_jump
                pred = query(tree[int(jump_loc):,],tmp_test_array)

            else:
                # If >=, go right branch:
                jump_loc = right_tree_jump
                pred = query(tree[int(jump_loc):,],tmp_test_array)

            return pred

        assert len(tmp_test_array.shape) == 2, "Make sure your test data is 2d array"
        result = []

        for i in range(tmp_test_array.shape[0]):
            inp = tmp_test_array[i,:].reshape(1,-1)
            result.append(query(self.tree,inp))
        return result  
        
        
        
    
    
    

#### As always load the toy dataset breast cancer classification, we will just use a small subset to test if the algorithm is working

In [3]:
X,y = load_breast_cancer(return_X_y=True)
X,y = X[:30,:5],y[:30,]

In [4]:
# Construct and Fit:
model = id3_tree(least_children_num = 2,verbose=False)
model.fit(X,y)


#### This is how our actual tree look like, spend some time to stare at it

* Each row is a node

* the rows with 3 nan are leaves, means no more children they have

* The format is [split_col, split_val, left_tree_relative_idx, right_tree_relative_idx]

In [5]:
model.tree

array([[ 2.    , 87.5   ,  1.    ,  4.    ],
       [ 4.    ,  0.1186,  1.    ,  2.    ],
       [    nan,  1.    ,     nan,     nan],
       [    nan,  0.    ,     nan,     nan],
       [    nan,  0.    ,     nan,     nan]])

#### We will cheat a little in here because we will just use training data to predict to check the bottom line that if it is working with training data

In [6]:
pred = model.predict(X)

In [7]:
import pandas
print(pandas.DataFrame([i for i in zip(pred,y)],columns=['in_sample_prediction','label']))

    in_sample_prediction  label
0                    0.0      0
1                    0.0      0
2                    0.0      0
3                    0.0      0
4                    0.0      0
5                    0.0      0
6                    0.0      0
7                    0.0      0
8                    0.0      0
9                    0.0      0
10                   0.0      0
11                   0.0      0
12                   0.0      0
13                   0.0      0
14                   0.0      0
15                   0.0      0
16                   0.0      0
17                   0.0      0
18                   0.0      0
19                   1.0      1
20                   1.0      1
21                   1.0      1
22                   0.0      0
23                   0.0      0
24                   0.0      0
25                   0.0      0
26                   0.0      0
27                   0.0      0
28                   0.0      0
29                   0.0      0


In [8]:
# Accuracy Function:
accuracy = lambda pred,y:   float(sum([pred[i]==y[i] for i in range(len(y))])) / len(y)

In [9]:
y_list = y.tolist()

#### Not bad,  it's getting 100% in-sample accuracy, means the tree is splitting correctly

In [10]:
accuracy(pred,y_list)

1.0

###  Now Fit with all data:

In [11]:
X,y = load_breast_cancer(return_X_y=True)


In [12]:
# Construct and Fit:
model = id3_tree(least_children_num = 3,verbose=False)
model.fit(X,y)


no better split...return mode


In [13]:
# Tree:
model.tree

array([[2.200e+01, 1.060e+02, 1.000e+00, 2.000e+01],
       [2.700e+01, 1.359e-01, 1.000e+00, 1.400e+01],
       [1.300e+01, 4.911e+01, 1.000e+00, 1.000e+01],
       [2.100e+01, 3.025e+01, 1.000e+00, 2.000e+00],
       [      nan, 1.000e+00,       nan,       nan],
       [1.100e+01, 8.927e-01, 1.000e+00, 2.000e+00],
       [      nan, 0.000e+00,       nan,       nan],
       [2.000e+01, 1.449e+01, 1.000e+00, 2.000e+00],
       [      nan, 1.000e+00,       nan,       nan],
       [0.000e+00, 1.338e+01, 1.000e+00, 2.000e+00],
       [      nan, 0.000e+00,       nan,       nan],
       [      nan, 1.000e+00,       nan,       nan],
       [4.000e+00, 9.387e-02, 1.000e+00, 2.000e+00],
       [      nan, 1.000e+00,       nan,       nan],
       [      nan, 0.000e+00,       nan,       nan],
       [2.100e+01, 2.795e+01, 1.000e+00, 4.000e+00],
       [2.800e+01, 3.630e-01, 1.000e+00, 2.000e+00],
       [      nan, 1.000e+00,       nan,       nan],
       [      nan, 0.000e+00,       nan,      

In [14]:
pred = model.predict(X)

In [15]:
import pandas
print(pandas.DataFrame([i for i in zip(pred,y)],columns=['in_sample_prediction','label']))

     in_sample_prediction  label
0                     0.0      0
1                     0.0      0
2                     0.0      0
3                     0.0      0
4                     0.0      0
5                     0.0      0
6                     0.0      0
7                     0.0      0
8                     0.0      0
9                     0.0      0
10                    0.0      0
11                    0.0      0
12                    0.0      0
13                    0.0      0
14                    0.0      0
15                    0.0      0
16                    0.0      0
17                    0.0      0
18                    0.0      0
19                    1.0      1
20                    1.0      1
21                    1.0      1
22                    0.0      0
23                    0.0      0
24                    0.0      0
25                    0.0      0
26                    0.0      0
27                    0.0      0
28                    0.0      0
29        

#### with all data we are getting 99.8% in-sample accuracy, nothing surprised here

In [16]:
y_list = y.tolist()
accuracy(pred,y_list)

0.9982425307557118

### Now, let's be more serious, and do some real data science, let's hold-out a part to do validation, to see how well the tree generalized:

In [17]:
X,y = load_breast_cancer(return_X_y=True)

In [18]:
# shuffling:
idx = [i for i in range(len(y))]
np.random.seed(1028)
np.random.shuffle(idx)

##### We use 20% data for validation, which is 113 examples

In [19]:
val_ratio = 0.2
val_num = int(len(y)*val_ratio)

print("We will be using {} validation examples".format(val_num))

We will be using 113 validation examples


In [20]:
X_train,X_valid = X[val_num:], X[:val_num]

y_train,y_valid = y[val_num:], y[:val_num]

In [21]:
print("Shapes:")
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

Shapes:
(456, 30)
(456,)
(113, 30)
(113,)


In [22]:
# Construct and Fit:
model = id3_tree(least_children_num = 3,verbose=False)
model.fit(X_train,y_train)


In [23]:
# Tree:
print(model.tree)

[[2.200e+01 1.177e+02 1.000e+00 2.400e+01]
 [2.700e+01 1.654e-01 1.000e+00 2.200e+01]
 [2.200e+01 1.017e+02 1.000e+00 1.000e+01]
 [1.300e+01 4.911e+01 1.000e+00 6.000e+00]
 [2.100e+01 3.337e+01 1.000e+00 2.000e+00]
 [      nan 1.000e+00       nan       nan]
 [2.100e+01 3.375e+01 1.000e+00 2.000e+00]
 [      nan 0.000e+00       nan       nan]
 [      nan 1.000e+00       nan       nan]
 [5.000e+00 6.601e-02 1.000e+00 2.000e+00]
 [      nan 0.000e+00       nan       nan]
 [      nan 1.000e+00       nan       nan]
 [2.100e+01 2.803e+01 1.000e+00 6.000e+00]
 [2.400e+01 1.426e-01 1.000e+00 2.000e+00]
 [      nan 1.000e+00       nan       nan]
 [4.000e+00 1.096e-01 1.000e+00 2.000e+00]
 [      nan 1.000e+00       nan       nan]
 [      nan 0.000e+00       nan       nan]
 [1.000e+01 3.860e-01 1.000e+00 4.000e+00]
 [1.000e+01 2.323e-01 1.000e+00 2.000e+00]
 [      nan 0.000e+00       nan       nan]
 [      nan 1.000e+00       nan       nan]
 [      nan 0.000e+00       nan       nan]
 [      nan

In [24]:
# Prediction:
pred = model.predict(X_valid)

In [25]:
import pandas
print(pandas.DataFrame([i for i in zip(pred,y_valid)],columns=['out_sample_prediction','pred']))

     out_sample_prediction  pred
0                      0.0     0
1                      0.0     0
2                      0.0     0
3                      0.0     0
4                      0.0     0
5                      0.0     0
6                      0.0     0
7                      0.0     0
8                      0.0     0
9                      0.0     0
10                     0.0     0
11                     0.0     0
12                     0.0     0
13                     1.0     0
14                     0.0     0
15                     0.0     0
16                     0.0     0
17                     0.0     0
18                     0.0     0
19                     1.0     1
20                     1.0     1
21                     1.0     1
22                     0.0     0
23                     0.0     0
24                     0.0     0
25                     0.0     0
26                     0.0     0
27                     0.0     0
28                     0.0     0
29        

In [26]:
y_valid_list = y_valid.tolist()
accuracy(pred,y_valid_list)

0.9026548672566371

### Not bad, we reached 90.3% out-of-sample accuracy with a single tree. That's why I'm a tree hugger =)

## Next: 

* Random Forest:  https://gecgithub01.walmart.com/y0w0252/Algorithms/blob/master/Machine_Learning_with_Numpy/All%20you%20need%20to%20know%20about%20Trees-2%20Random%20Forest.ipynb